In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

# Datasets
customers = pd.read_csv("Customers.csv")
transactions = pd.read_csv("Transactions.csv")
products = pd.read_csv("Products.csv")

# Merging transactions with products to include category information
merged_data = transactions.merge(products[['ProductID', 'Category']], on='ProductID')

# Transaction data for each customer
customer_features = merged_data.groupby('CustomerID').agg(
    TotalSpending=('TotalValue', 'sum'),
    TotalTransactions=('TransactionID', 'count'),
    TotalQuantity=('Quantity', 'sum'),
    PreferredCategory=('Category', lambda x: x.value_counts().idxmax())
).reset_index()

# Merging customer details
customer_features = customer_features.merge(customers, on='CustomerID')

# Encoded PreferredCategory and Region as numerical features
customer_features = pd.get_dummies(customer_features, columns=['PreferredCategory', 'Region'])

# Normalizing
scaler = MinMaxScaler()
normalized_features = customer_features.drop(columns=['CustomerID', 'CustomerName', 'SignupDate'])
normalized_features = pd.DataFrame(scaler.fit_transform(normalized_features), columns=normalized_features.columns)

# Computing similarity matrix
similarity_matrix = cosine_similarity(normalized_features)

# Mapping customer IDs to indices
customer_indices = customer_features['CustomerID']

# Recommendations for the first 20 customers
recommendations = []
for i in range(20):
    similar_customers = sorted(
        [(customer_indices[j], similarity_matrix[i, j]) for j in range(len(similarity_matrix)) if i != j],
        key=lambda x: x[1],
        reverse=True
    )[:3]
    recommendations.append({
        'CustomerID': customer_indices[i],
        'Lookalikes': str([(rec[0], round(rec[1], 4)) for rec in similar_customers])
    })

# CSV
recommendations_df = pd.DataFrame(recommendations)
recommendations_df.to_csv("sammed_barwade_Lookalike.csv", index=False)

print("Lookalike.csv file saved to sammed_barwade_Lookalike.csv in the required format.")


Lookalike.csv file saved to sammed_barwade_Lookalike.csv in the required format.
